In [11]:
import sqlite3
conn = sqlite3.connect('example.db')

c = conn.cursor()

c.execute('''DROP TABLE person''')

c.execute('''CREATE TABLE person (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)''')

c.execute('''DROP TABLE address''')

c.execute('''
        CREATE TABLE address
        (id INTEGER PRIMARY KEY ASC, 
        street_name varchar(250), 
        street_number varchar(250),
        post_code varchar(250) NOT NULL, 
        person_id INTEGER NOT NULL,
        FOREIGN KEY(person_id) REFERENCES person1(id))
        ''')

c.execute('''
        INSERT INTO person values(1, 'pythoncentral')
        ''')
c.execute('''
        INSERT INTO address VALUES(1, 'python road', '1', '00000', 1)
        ''')
conn.commit()
conn.commit()

In [12]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM person')
print(c.fetchall())
c.execute('SELECT * FROM address')
print(c.fetchall())
conn.close()

[(1, 'pythoncentral')]
[(1, 'python road', '1', '00000', 1)]


In [16]:
import os, sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

Base = declarative_base()

class Person(Base):
    __tablename__ = 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)
    
engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.create_all(engine)

In [18]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_declarative import Address, Base, Person

engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()

new_person = Person(name='new_person')
session.add(new_person)
session.commit()

new_address = Address(post_code='00000', person=new_person)
session.add(new_address)
session.commit()

In [20]:
from sqlalchemy_declarative import Address, Base, Person
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine
from sqlalchemy.orm import sessionmaker
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()
session.query(Person).all()


In [21]:
person = session.query(Person).first()
person.name


'new_person'

In [22]:
session.query(Address).filter(Address.person == person).all()


In [23]:
session.query(Address).filter(Address.person == person).one()


In [24]:
address = session.query(Address).filter(Address.person == person).one()
address.post_code

'00000'